# Exploring NYC Housing Costs, An Introduction to Python for Data Science

## This demo uses a data set available from NYC Open Data (https://nycopendata.socrata.com/) to showcase how to use python for data science. In particular, it will be focused on loading tabular data, data munging, plotting, calculating descriptive statistics, modeling data, and working with time series data

In [ ]:
# Importing required libraries

%matplotlib inline  
import os
import matplotlib.pyplot as plt
import datetime
from os import listdir
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import cross_val_predict
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor

###  1) Introducing Data Frames (Importing, Indexing)

In [ ]:
# Loading in a single Excel file into a data frame using pandas. This is just one of the files
# we will need to reed in.  Let's read it in as a test. Note: file has a 4 line header.

dataTest = pd.read_excel('Datafiles/2009_brooklyn.xls', header=4)

In [ ]:
# Displaying lines from data frame (note: columns are labeled from data in file)

dataTest

In [ ]:
# There are multiple ways to index into a pandas data frame.
# You can index an entire column using the dot notation.

dataTest.ADDRESS  # also can use data['ADDRESS']

In [ ]:
# You can retrieve specific columns and rows using .ix. This is flexible because ix supports both 
# label and integer position indexing. However, this can get confusing if you use integers to label either your
# rows for columns.  If you want to specify that you are  only using labels, you can index using .loc and if you 
# you want to only integer positions then you an use .iloc. 

dataTest.ix[:5,'ADDRESS']  # note this indexing will return a data frame or series 

In [ ]:
# If you want the underlying value in a data frame - you can use .values

dataTest.ix[:5,'ADDRESS'].values

In [ ]:
# Getting list of housing files to import

fileList = listdir('Datafiles')   # getting files/directories in the folder
fileListHousing = [x for x in fileList if '.xls' in x]  # extracting file names that contain .xls

fileListHousing

In [ ]:
# Pandas automatically reads in our column names from the line following the header lines. We can use that
# fact to determine if our file either has a 4 or a 3 line header

dataTest.columns[0]

In [ ]:
dataAll = pd.DataFrame()

# Loop through all files and append to precious read in data. Files either have a header of 3 lines of 4 lines. 
# To work around this, we try 4 header lines.  If that does not work, we use 3 header lines.

for file in fileListHousing:
    dataRead = pd.read_excel('Datafiles/' + file, header=4) 
    if dataRead.columns.values[0] != 'BOROUGH':
        dataRead = pd.read_excel('Datafiles/' + file, header=3)  
    dataAll = dataAll.append(dataRead)

### 2) Cleaning and Plotting Data

In [ ]:
# pandas has built-in statistical methods that work directly on data frames. 
# Here is a handy list in the documenation:

dataAll.describe()

In [ ]:
# Cleaning the data by removing the NaNs and unrealistic values for other variables. 
# Note: you use DataFrame.clip, if you want to threshhold your data

dataClean = dataAll.dropna(subset ={'SALE PRICE'})

idx = dataClean['SALE PRICE'] > 1000  
idx2 = dataClean['LAND SQUARE FEET'] > 0  
idx3 = dataClean['GROSS SQUARE FEET'] > 0  
idx4 = dataClean['RESIDENTIAL UNITS'] ==  1
idx5 = dataClean['COMMERCIAL UNITS'] ==  0
idx6 = dataClean['BOROUGH'] > 0
idx7 = dataClean['ZIP CODE'] > 0 

dataClean['ZIP CODE'] = dataClean['ZIP CODE'].apply(str)  # make zipcode into a string

dataClean  = dataClean[idx & idx2 & idx3 & idx4  & idx5 & idx6 & idx7]

In [ ]:
dataClean

In [ ]:
# Apply function to a column.  There are build in functions that automatically work with data frames.  If you
# do not specify a column, the function will be run on the entire data frame, where possible.

dataClean['SALE PRICE'].mean()

In [ ]:
# Can use apply to apply a function to a column when it is not supported directly on a data frame. Can add
# the resuling column to a data frame. 

dataClean['LOG PRICE'] = dataClean['SALE PRICE'].apply(np.log10)

In [ ]:
# Built in plotting is available for data frames.  This is an easy (quick & dirty) way to view your data. 
# You can make more elaborate or publication quality graphs by using matplotlib or seaborn directily. 

dataClean['LOG PRICE'].hist()

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 6))

# Draw a boxplot with a narrower bandwidth than the default
ax = sns.boxplot(data=dataClean, x='BOROUGH', y='SALE PRICE')
ax.set_yscale("log")
bNames = ['Manhattan','Bronx','Brooklyn','Queens','Staten Island']
ax.set_xticklabels(bNames);

### 3) Grouping and Reshaping Data

In [ ]:
# Grouping allows you to apply functions to a group of data.  In this case, let's look at mean price by zipcode. 

grpBorough = dataClean.groupby(by='BOROUGH')
byBorough = grpBorough['SALE PRICE'].describe()
byBorough

In [ ]:
# Can unstack the data (similar to using some functionality of pivot tables in Excel), Stacking is also 
# available as well as the method pivot_table with allows for full pivot table functionality 

byBorough.unstack(1)

### 4) Creating Geographical Maps

#### For reference here is map of NYC with boroughs labeled

<img src="Images/New_York_City_District_Map_2.png",width=200,height=200>

In [ ]:
# Plotting mean price on a map of Brooklyn
# Calculating mean price by zipcode

grpZip = dataClean.groupby(by='ZIP CODE')
meanByZip = grpZip['SALE PRICE'].mean()

In [ ]:
# Plotting the base map
fig, ax2 = plt.subplots(1,1,figsize=(12, 12))
m = Basemap(llcrnrlon=-74.5,llcrnrlat=40.45,urcrnrlon=-73.5,urcrnrlat=40.95,
            projection='lcc',lat_1=39.5,lat_2=41,lon_0=-74)

# Read in the zipcode map shapefile
shp_info = m.readshapefile(cwd + '/Mapfiles/cb_2013_us_zcta510_500k','zipCode',drawbounds= True, linewidth = 0.05)

# Getting a list of zip codes we want to plot
zipcode_list = meanByZip.index.values
zipcode_list = zipcode_list.tolist()

# Setting the colormap
cmap = plt.cm.get_cmap('viridis')  # by default levels 0-256

cindex= []
zipcodes = []

# Calculating the max, min log sale price values, will be used to normalize price to index into colorscale
maxZip = np.float64(meanByZip.values.max())
maxZipLog = np.log10(maxZip)
minZip = np.float64(meanByZip.values.min())
minZipLog = np.log10(minZip)

# Find zipcodes for new shape in shape file
for shapedict in m.zipCode_info:
    # cycle through zipcode
    zipcode = shapedict['ZCTA5CE10']
    zipcodes.append(zipcode)

ax1 = plt.gca() # get current axes instance

# For each shape in shape file, plot polygon segments.  Color is picked by indexing into colormap using
# normalized cost, colormap by default has 256 levels,

for nshape,seg in enumerate(m.zipCode):
    if zipcodes[nshape] in zipcode_list:
        cost = np.log10(meanByZip.loc[zipcodes[nshape]])
        cindex = int(((cost - minZipLog)/(maxZipLog - minZipLog))*257)
        color = cmap(cindex)[:3]
        poly = Polygon(seg,facecolor = color,  edgecolor= 'k')
        ax1.add_patch(poly)
        
plt.title('Mean 1-Bedroom House Prices: 2003-2009')       

# Create a scaled colorbar that gives the range of values

sm = plt.cm.ScalarMappable(cmap=cmap)
sm.set_array(np.linspace(minZipLog,maxZipLog,257))
cb = plt.colorbar(sm, shrink=.5, pad = 0.1)
cb.ax.set_title('Log10 of Sale Price');


### 5) Merging Datasets 

In [ ]:
# Merging New Data into out DataSet

# Let's load in population and plot that verus mean price per zipcode
# Unlike before, let's specify zip code as a string and not a number.

popData = pd.read_csv('Datafiles/2010Census.csv',dtype = {'Zip Code ZCTA':str})
popData

In [ ]:
meanbyZip2 =  grpZip.mean()

meanAllbyZip = meanbyZip2.merge(popData,how='inner', left_index='TRUE', right_on='Zip Code ZCTA')
meanAllbyZip

In [ ]:
# Can make a scatter plot to see the influence of population on housing prices for the different boroughs

fig, ax = plt.subplots(1,1,figsize=(8, 4))
cmp = plt.cm.get_cmap('Set2',5)

ax.scatter(meanAllbyZip['2010 Census Population'], meanAllbyZip['SALE PRICE'],c = meanAllbyZip['BOROUGH'], 
           s = 80, cmap = cmp,alpha = 0.5)

ax.set_title('Sale Price vs Population (Average by Zip Code)');
ax.set_yscale('log')
ax.set_xlim([0,120000]);

### 6) Working with Time Series Data

In [ ]:
# So far we have dealt with our data as if does not have a time component, if we use the date of purchase as the 
# index, we can use our data frame like a timeseries. 

# First, we need to convert our SALE DATA to a datetime 

dataClean['SALE DATE'] = pd.to_datetime(dataClean['SALE DATE'])

# Next, set SALE DATE as our index.  And now we can use the data frame as a time series. 

dataTime = dataClean.set_index(dataClean['SALE DATE'].values)

In [ ]:
dataTime

In [ ]:
# Can now index using a date, or range of dates

dataTime['2004-12-01':'2005-12-01']

In [ ]:
# We can also resample (and apply a function to the data in the sample window). In this
# case let's resample every month, and get the mean price.

dataTimebyMonth = dataTime['SALE PRICE'].resample('1m').mean()
dataTimebyMonth[1:10]

In [ ]:
# We actually want this divided by borough. So, we can group and then apply our resampling. 

dataCTbyBorough = dataTime.groupby('BOROUGH')

g = lambda x: x.resample('1m').mean()  # define anonymous function to resample each group of data 

# Or could define a regular function ....

def galt(x):
    xresample = x.resample('1m').mean()
    return xresample

# Apply function to each group

#salebyMonth = dataCTbyBorough['SALE PRICE'].apply(galt)

salebyMonth = dataCTbyBorough['SALE PRICE'].apply(g)
salebyMonth = salebyMonth.unstack(0)  # unstack so we have time as the rows and borough as the columns

salebyMonth[1:10]

In [ ]:
# Plot results, timeseries data frames are supported by the built-in plot method.  
# Note that the build in methods do offer several options for customization. 

f, ax4 = plt.subplots(1,1,figsize=(8, 4))
salebyMonth.plot(ax = ax4, logy = True, title = 'Mean Sale Price in Time', colormap = 'Dark2')
ax4.legend(labels=bNames,loc = 'upper left')

### 7) Modeling Data

In [ ]:
# There are many different ways to model this data.  There is clearly a time component. For the sake of 
# simplicity since this demo is focused on showing how to use the functionality and on the getting the BEST model, 
# we are just going to take data from a fixed year (2009)

# Extracting data from 2009
data2009 = dataTime['2009-01-01':'2009-12-31']

In [ ]:
# Let's clean out the variables that are constant (Commerical Units, etc) and keep variables that may be of
# importance to our model. For simplicity again, we are going to consider Borough, Zipcode, Land Square Feet, 
# Gross Square Feet

Xtotal = data2009.ix[:,{'BOROUGH','ZIP CODE','GROSS SQUARE FEET','LAND SQUARE FEET'}]
ytotal = data2009.ix[:,{'LOG PRICE'}]

In [ ]:
# We need to treat as categorical variables, since even though
# they are numbers, they represent categories and not numbers (i.e. 1 isn't greater than 5 for borough). Pandas
# can let us convert them to dummy variables (convert using onehot encoding). 

XtotalDMY = pd.get_dummies(Xtotal, columns={'BOROUGH','ZIP CODE'})

In [ ]:
XtotalDMY

In [ ]:
# Standardize our data. 

scaler = preprocessing.StandardScaler().fit(XtotalDMY)
Xscaled = scaler.transform(XtotalDMY) 

In [ ]:
# Fitting the data using linear regression, then plotting the test data vs the predicted values. 

# Create linear regression object
regr = linear_model.LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(Xscaled, ytotal, test_size=0.2, random_state=0)

# Train the model using the training sets
regr.fit(X_train, y_train)
predicted = regr.predict(X_test)
             
f, ax = plt.subplots(1,1,figsize=(8, 4))
ax.scatter(y_test, predicted)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')

In [ ]:
# Fitting the data using random forest, then plotting the test data vs the predicted values. In
# this case we are using defaults for our random forest, but there is a nice example on how to tweak
# the values for best performace for your particular data here--
# http://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html#sphx-glr-auto-examples-ensemble-plot-ensemble-oob-py

regr_rf = RandomForestRegressor() 
regr_rf.fit(X_train, y_train.values.ravel())

# Predict on new data
predicted_rf = regr_rf.predict(X_test)

f, ax = plt.subplots(1,1,figsize=(8, 4))
ax.scatter(y_test, predicted_rf)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')